In [1]:
import logging
import threading
from time import sleep, time
import random

from kazoo.protocol.paths import join

logging.basicConfig()

from kazoo.client import KazooClient

In [2]:
class Client(threading.Thread):
    def __init__(self, root: str, _id: int):
        super().__init__()
        self.url = f'{root}/{_id}'
        self.root = root
        self.id = _id

    def run(self):
        zk = KazooClient()
        zk.start()

        value = b'commit' if random.random() > 0.5 else b'abort'
        print(f'Client {self.id} request {value.decode()}')
        zk.create(self.url, value, ephemeral=True)
        
        @zk.DataWatch(self.url)
        def watch_myself(data, stat):
            if stat.version != 0:
                print(f'Data of client {self.id} was set to {data.decode()}')

        sleep(5)

        zk.stop()
        zk.close()

In [3]:
class Coordinator():
    def main(self):
        coordinator = KazooClient()
        coordinator.start()

        if coordinator.exists('/coordinator'):
            coordinator.delete('/coordinator', recursive=True)

        coordinator.create('/coordinator')
        coordinator.create('/coordinator/tx')
        number_of_clients = 5

        def make_decision():
            clients = coordinator.get_children('/coordinator/tx')
            n_commit = 0
            n_abort = 0
            for client in clients:
                n_commit += int(coordinator.get(f'/coordinator/tx/{client}')[0] == b'commit')
                n_abort += int(coordinator.get(f'/coordinator/tx/{client}')[0] == b'abort')

            target = b'commit' if n_commit > n_abort else b'abort'
            for client in clients:
                coordinator.set(f'/coordinator/tx/{client}', target)

        @coordinator.ChildrenWatch('/coordinator/tx')
        def watch_clients(clients):
            if len(clients) < number_of_clients:
                print('Waiting for the others.', clients)
            elif len(clients) == number_of_clients:
                print('Gathering voices and making final decision')
                make_decision()

        root = '/coordinator/tx'
        for i in range(5):
            p = Client(root, i)
            p.start()

In [4]:
Coordinator().main()

Waiting for the others. []
Client 0 request abort
Client 1 request abort
Client 2 request commit
Client 3 request abort
Client 4 request abort
Waiting for the others. ['0', '1', '2', '3']
Gathering voices and making final decision
Data of client 0 was set to abort
Data of client 1 was set to abort
Data of client 2 was set to abort
Data of client 3 was set to abort
Data of client 4 was set to abort
Waiting for the others. ['3', '4']
Waiting for the others. []
